In [ ]:
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import pickle

from set_home_directory import get_project_root_homedir_in_sys_path
project_root, main_dir = get_project_root_homedir_in_sys_path("inter_areal_predictability")
if project_root is None:
    raise RuntimeError(f"Project root not found: ensure a folder named '{project_root}' exists in one of the sys.path entries.")
print("Project root found:", project_root)
os.chdir(project_root)



import sys
sys.path.insert(0,os.path.join(main_dir,'utils/'))
sys.path.insert(0,main_dir)

from utils.fig_3_functions import plot_activitytype_by_direction_densities, p_to_stars
from utils.fig_6_functions import pairwise_within_x_using_your_tests

save_figs = False

In [ ]:
fig_dir = os.path.join(main_dir, 'results/paper_figures/revisions/')
monkey_name='L'
monkey=monkey_name
monkey_stats_path = os.path.join(main_dir, 'results/fig_2',f'monkey_{monkey_name}_stats.pkl')
with open(monkey_stats_path, 'rb') as f:
	monkey_stats = pickle.load(f)

### functions

In [ ]:
all_ini_stim_offs = {'SNR': 400, 'SNR_spont': 300, 'RS': None,
                    'RS_open':None, 'RS_closed': None, 
                    'RF_thin':1000, 'RF_large':1000, 'RF_thin_spont':300, 
                    'RF_large_spont':300}
#depending on the dataset type, there are different times of autocorrelation to mitigate
all_frames_reduced = {'SNR': 5, 'SNR_spont': 5, 'RS': 20, 
                    'RS_open':20, 'RS_closed': 20, 
                    'RF_thin':25, 'RF_large':25, 'RF_thin_spont':25, 'RF_large_spont':25}

In [ ]:
def get_property_dataset_type_monkey(input_string):
    if 'spont' in input_string:
        return input_string.replace('_spont','')
    elif 'RS' in input_string:
        return 'SNR'
    else:
        return input_string 
    
def get_reli_condition(input_string):
    if 'spont' in input_string:
        return input_string.replace('_spont','')
    elif 'RS' in input_string:
        return 'SNR'
    else:
        return input_string
    
def make_monkey_df(monkey_stats_, dataset_types=['SNR', 'RF_thin', 'RF_large','RS']):
	"""
	Create a DataFrame from the provided monkey statistics data for different dataset types.

	Args:
	- monkey_stats_ (_type_): Monkey statistics data.
	- dataset_types (list, optional): List of dataset types. Defaults to ['SNR', 'RF_thin', 'RF_large'].

	Returns:
	- pandas.DataFrame: DataFrame containing the collected monkey data.
	"""
	data = []
	for dataset_type in dataset_types:
		# print(dataset_type)
		if 'spont' in dataset_type:
			act_type = 'gray screen'
		elif 'RS' in dataset_type:
			act_type = 'lights off'
		else:
			act_type = 'stimulus'
		for date, areas_data in monkey_stats_[dataset_type].items():
			# skip the dates with no V4 electrodes  
			if date in ['140819', '150819', '160819','250717']:
						continue
			print(date)
			for area, values in areas_data.items():
				# print(area)
				if area =='V4':
					direction= 'V1→V4'
				else:
					direction = 'V4→V1'
				# Get the split-half orrelation values for the current area
				split_half_rs = monkey_stats_[get_property_dataset_type_monkey(dataset_type)][date][area]['split_half_r']
				# Get the SNR values for the current area
				SNRs = monkey_stats_[get_property_dataset_type_monkey(dataset_type)][date][area]['SNR_meanspont']
				for w_size in [10,25,50,100,200]:
					for cell_n, (split_half_r, snr, evar, null_evar) in enumerate(zip(split_half_rs, SNRs,values[f'evars_{w_size}'],values[f'evars_{w_size}_null'])):
						# Append data for the actual experiment (control_shuffle = False)
						data.append({
							'Dataset Type': dataset_type,
							'Activity Type': act_type,
							'Date':date,
							'Area': area,
							'EV': evar,
							'SNR': snr,
							'Split-half r': split_half_r,
							'control_shuffle':False,
							'Cell Number': cell_n,
							'bin size (ms)': w_size,
							'Direction': direction
							
						})
						# Append data for the shuffled experiment (control_shuffle = True
						data.append({
							'Dataset Type': dataset_type,
							'Activity Type': act_type,
							'Date': date,
							'Area': area,
							'EV': null_evar,
							'SNR': snr,
							'Split-half r': split_half_r,
							'control_shuffle':True, 
							'Cell Number': cell_n,
							'Direction': direction,
							'bin size (ms)': w_size
						})
	# Create a DataFrame from the flattened data
	df_monkey_all = pd.DataFrame(data)
	return df_monkey_all

def make_monkey_df_directionality(monkey_stats_, dataset_types=['SNR','RF_thin','RF_large','RS'], verbose=False):
	"""Creates a DataFrame for monkey directionality data.

	This function iterates over the provided mouse statistics data and constructs a DataFrame containing directionality
	information for each mouse, area, and permutation. It extracts relevant information such as date, area,
	direction, reliability, SNR, and maximum correlation value.

	Args:
		mouse_stats_ (dict): Dictionary containing monkey statistics data.
		dataset_types (list, optional): List of dataset types. Defaults to ['SNR','RF_thin','RF_large'].

	Returns:
		pandas.DataFrame: DataFrame containing directionality data.
	"""
	data = []
	for dataset_type in dataset_types:
		if 'spont' in dataset_type:
			act_type = 'gray screen'
		elif 'RS' in dataset_type:
			act_type = 'lights off'
		else:
			act_type = 'stimulus'
		for date, areas_data in monkey_stats_[dataset_type].items():
			if date in ['140819', '150819', '160819', '250225']:
				continue
			list_of_unique_indices = []
			seeds_to_ommit = []
			for s in range(10):
				for area_ in ['V1','V4']:
					if 'big_chosen_indices' in list(monkey_stats_[get_reli_condition(dataset_type)][date][area_].keys()):
								ordered_indices = np.sort(monkey_stats_[get_reli_condition(dataset_type)][date][area_]['big_chosen_indices'][s])
								if any(np.sum(ordered_indices == x) == len(ordered_indices) for x in list_of_unique_indices):
									if verbose:
										print(f'skipping {dataset_type} {date} {area_} seed {s}, indices already used')
									seeds_to_ommit.append(s)
								else:
									# print([np.sum(ordered_indices == x)/ len(ordered_indices) for x in list_of_unique_indices])
									list_of_unique_indices.append(ordered_indices)
                                
			for area, values in areas_data.items():
				if 'directionality_evars_5' not in list(values.keys()):
					if verbose:
						print(f'skipping {dataset_type} {date} {area}, no directionality evars found')
					continue
				if len(values['directionality_evars_5'])==1:
					if np.isnan(values['directionality_evars_5'][0].any()):
						if verbose:
							print('skipping directionality, none was recorded')
						continue
				split_half_rs = monkey_stats_[get_reli_condition(dataset_type)][date][area]['split_half_r']
				SNRs = monkey_stats_[get_reli_condition(dataset_type)][date][area]['SNR_meanspont']
				if verbose:
					print(dataset_type, date, area)
				max_corr_vals = monkey_stats_[dataset_type][date][area]['max_corr_val']
				n_seeds = len(values['directionality_evars_5'])
				for w_size in [5,10,25,50,100,200]:
					for s in range(n_seeds):
						if s in seeds_to_ommit:
							continue
						direction_evars = values[f'directionality_evars_{w_size}'][s]
						direction_evars_null = values[f'directionality_evars_{w_size}_null'][s] if (dataset_type=='RS_open')&(date=='260225') else values['directionality_evars_null'][s]
						if area =='V1':
							direction = 'V4→V1'
							if 'big_chosen_indices' in list(monkey_stats_[get_reli_condition(dataset_type)][date][area].keys()):
								v1_indices = list(monkey_stats_[get_reli_condition(dataset_type)][date][area]['big_chosen_indices'][s])
							elif 'small_chosen_indices' in list(monkey_stats_[get_reli_condition(dataset_type)][date][area].keys()):
								v1_indices = monkey_stats_[get_reli_condition(dataset_type)][date][area]['small_chosen_indices']
							else:
								print(f'no V1 indices found for {dataset_type} {date} {area}. skipping this condition type and date')
								break
							chosen_split_half_rs = split_half_rs[v1_indices]
							chosen_SNRs= SNRs[v1_indices]
							chosen_max_corr_vals = max_corr_vals[v1_indices]
							neuron_indices = v1_indices
						else:
							direction = 'V1→V4'
							if 'big_chosen_indices' in list(monkey_stats_[get_reli_condition(dataset_type)][date][area].keys()):
								v4_indices = monkey_stats_[get_reli_condition(dataset_type)][date][area]['big_chosen_indices'][s]
							elif 'small_chosen_indices' in list(monkey_stats_[get_reli_condition(dataset_type)][date][area].keys()):
								v4_indices = monkey_stats_[get_reli_condition(dataset_type)][date][area]['small_chosen_indices']
							else:
								print(f'no V1 indices found for {dataset_type} {date} {area}. skipping this condition type and date')
								break
							chosen_split_half_rs = split_half_rs[v4_indices]
							chosen_SNRs = SNRs[v4_indices]
							chosen_max_corr_vals = max_corr_vals[v4_indices]
							neuron_indices = v4_indices
						if verbose:
							print(f'Processing {dataset_type} {date} {area} seed {s}, {len(neuron_indices)} neurons')

						for n, (split_half_r, snr, max_corr_val, direction_evar, direction_evar_null, neuron_index) in enumerate(zip(chosen_split_half_rs, 
																														chosen_SNRs, chosen_max_corr_vals, 
																														direction_evars, 
																														direction_evars_null,
																														neuron_indices)):
							data.append({
								'Dataset Type': dataset_type,
								'Activity Type': act_type,
								'Date': date,
								'Area': area,
								'Direction':direction,
								'EV': direction_evar,
								'SNR': snr,
								'Split-half r': split_half_r,
								'max corr. val': max_corr_val,
								'control_shuffle':False,
								'Permutation':s, 
								'Neuron index': neuron_index,
								'bin size (ms)': w_size
							})
							data.append({
								'Dataset Type': dataset_type,
								'Activity Type': act_type,
								'Date': date,
								'Area': area,
								'Direction':direction,
								'EV': direction_evar_null,
								'SNR': snr,
								'Split-half r': split_half_r,
								'max corr. val': max_corr_val,
								'control_shuffle':True, 
								'Permutation':s,
								'Neuron index': neuron_index,
								'bin size (ms)': w_size
							})
	# Create a DataFrame from the flattened data
	df_monkey_all = pd.DataFrame(data)
	return df_monkey_all





In [ ]:
fontsize=6
timescale_main_plot_args = {
	'figsize': (4.5,1.3),
	'tick_params':{'axis':'both','labelsize':fontsize,'width':0.5,'length':2, 'pad':1},
	'xlabel_params':{'labelpad':0,'fontsize':fontsize},
	'ylabel_params':{'labelpad':0,'fontsize':fontsize},
	'sup_title_args':{'y':1.05,'fontsize':fontsize*1.2},
	'supplots_adjust':{'wspace':0.1,'hspace':0.55},
	'ax_title_args':{'fontsize':fontsize*1.1,'y':0.97},
	'xticklabel_args':{'fontsize':fontsize},
}

### Monkey L plot EV timescales 

In [ ]:
monkey_name='L'
monkey=monkey_name
monkey_stats_path = os.path.join(main_dir, 'results/fig_2',f'monkey_{monkey_name}_stats.pkl')
with open(monkey_stats_path, 'rb') as f:
	monkey_stats = pickle.load(f)
df_monkey = make_monkey_df(monkey_stats, dataset_types=['SNR','RF_thin','RF_large','RS','SNR_spont','RS_open','RS_closed'])
filt_df_monkey =df_monkey[(df_monkey['Split-half r']>0.8)&(df_monkey.SNR>=2)].reset_index(drop=True)

In [ ]:
# Fig Supp 4A
fontsize=6
activity_types_ordered = ['stimulus','gray screen','lights off']
x = 'Activity Type'
hue = 'bin size (ms)'
y = 'EV'
hue_order = [10,25,50,100,200]  # or strings to match your data

plot_df = filt_df_monkey[filt_df_monkey['control_shuffle']==False].copy()

# stats per x label, correction within x (Holm by default)
res = pairwise_within_x_using_your_tests(
    plot_df, x=x, hue=hue, y=y,
    x_order=activity_types_ordered, hue_order=hue_order,
    compare='all',          # each bin vs the first bin (5 ms)
    mc_method='holm',
    perm_t=True, perm_type='ind', hierarchical=False,
    central_tendency='median', num_permutations=10000,
    mouse_or_date='Mouse Name'
)

centers = np.arange(len(activity_types_ordered))
n_h = len(hue_order)
# approximate seaborn offsets for grouped violins
width = 0.83
step = width / n_h if n_h > 0 else 0
start = -width/2 + step/2
offsets = [start + k*step for k in range(n_h)]
hue_to_offset = {h: offsets[i] for i, h in enumerate(hue_order)}


fig, ax = plt.subplots(1,1, figsize=timescale_main_plot_args['figsize'])

sns.violinplot(
    data=plot_df, x=x, y=y, hue=hue,
    order=activity_types_ordered, hue_order=hue_order,
    ax=ax, linewidth=0.5,
    inner_kws={'box_width':2, 'marker':'_', 'markersize':3, 'markeredgewidth':1, 'alpha':0.7},
    cut=0
)

for x_idx, x_i in enumerate(activity_types_ordered):
	for h1, h2 in zip(hue_order[1:], hue_order[:-1]):
		pval = res[x_i]['p_adj'][res[x_i]['pairs'].index((h1,h2))] if (h1,h2) in res[x_i]['pairs'] else (res[x_i]['p_adj'][res[x_i]['pairs'].index((h2,h1))] if (h2,h1) in res[x_i]['pairs'] else None)
		stars =p_to_stars(pval)
		subdf = plot_df[(plot_df[x]==x_i)&(plot_df[hue].isin([h1,h2]))]
		x_center = x_idx + 0.5*(hue_to_offset[h1] + hue_to_offset[h2])
		y_pos = subdf['EV'].max()*0.97	
		if stars == 'n.s.':
			fontsize_=fontsize*0.8
		else:
			fontsize_=fontsize
		ax.text(x_center, y_pos, stars, ha="center", va="top", fontsize=fontsize_)
sns.despine()

ax.legend(title='Bin size (ms)', loc=(1,0.3), fontsize=fontsize*0.8, title_fontsize=fontsize*0.8)
x_tick_labels = ['Stimulus', 'Gray screen', 'Lights off']
ax.set_xticklabels(x_tick_labels, **timescale_main_plot_args['xticklabel_args'])
ax.set_ylabel('EV', **timescale_main_plot_args['ylabel_params'])
ax.set_xlabel('')
ax.tick_params(**timescale_main_plot_args['tick_params'])
ax.legend_.get_frame().set_linewidth(0.5)
data = filt_df_monkey[filt_df_monkey['control_shuffle']==True]['EV']
per_25 = np.percentile(data.values, 25)
per_75 = np.percentile(data.values, 75)
ax.axhspan(per_25, per_75, alpha=0.3, color='blue', label='shuffle\ncontrol IQR',
		linewidth=0,
		)
ax.spines[:].set_linewidth(0.3)
if save_figs is True:
	fig.savefig(fig_dir + f'R2.6_monkey_predictions_all_bins_dataset_types.pdf', bbox_inches='tight', transparent=True)

### Monkey L plot directionality across timescales

In [ ]:
fontsize=6
timescale_dir_main_plot_args = {
	'figsize': (5,1.3),
	'tick_params':{'axis':'both','labelsize':fontsize,'width':0.5,'length':2, 'pad':1},
	'xlabel_params':{'labelpad':0,'fontsize':fontsize},
	'ylabel_params':{'labelpad':0,'fontsize':fontsize},
	'sup_title_args':{'y':1.05,'fontsize':fontsize*1.2},
	'supplots_adjust':{'wspace':0.1,'hspace':0.55},
	'ax_title_args':{'fontsize':fontsize*1.1,'y':0.97},
	'xticklabel_args':{'fontsize':fontsize},
}

In [ ]:
# Fig Supp 4C
monkey_name = 'L'
monkey=monkey_name

monkey_stats_path = os.path.join(main_dir, 'results/fig_3',f'monkey_{monkey_name}_stats.pkl')
with open(monkey_stats_path, 'rb') as f:
	monkey_stats = pickle.load(f)

df_monkey_directionality = make_monkey_df_directionality(monkey_stats, dataset_types=['SNR','RF_thin','RF_large','SNR_spont', 'RS_open','RS_closed'], verbose=False)
filt_df_monkey_directionality = df_monkey_directionality[(df_monkey_directionality.control_shuffle==False)].reset_index(drop=True)


fontsize=6
#'#136a66','#72BEB7','#B6E3DF','#a85959','#EDAEAE', '#f6d6d6
areas_palette = ['#72BEB7','#EDAEAE']
activity_types_ordered = ['stimulus','gray screen','lights off']
x_tick_labels = ['5','10','25','50','100','200']
x = 'bin size (ms)'
hue = 'Direction'
neuron_property = 'EV'
hue_order=['V1→V4','V4→V1']
fig, axes = plt.subplots (ncols=3, figsize=timescale_dir_main_plot_args['figsize'], sharey=True)
axes = axes.flatten()
# plot violin plot and change inner quartile widths using the inner_kws argument
for a, (act_type, ax) in enumerate(zip(activity_types_ordered, axes)):
	mini_df = df_monkey_directionality[df_monkey_directionality['Activity Type']==act_type]
	order = [int(i) for i in mini_df[x].unique()]
	_, _, pvals = plot_activitytype_by_direction_densities(
		mini_df, neuron_property=neuron_property, x=x, hue=hue,
		neuron_property_label="EV", ax=ax,
		order=order, hue_order=hue_order,
		fontsize=fontsize,
		linewidth=0,
	)
	ax.set_xlabel('Bin size (ms)', fontsize=fontsize, labelpad=0)
	ax.tick_params(axis='y', labelsize=fontsize)
	ax.set_title(act_type.capitalize(), fontsize=fontsize, y=0.9)
	if a ==0:
		ax.legend(title='', loc=(0.05,0.7), fontsize=fontsize*0.8, title_fontsize=fontsize*0.8)
		ax.legend_.get_frame().set_linewidth(0.2)	
	else:
		ax.legend_.remove()

plt.subplots_adjust(wspace=0.05, hspace=0.5)
if save_figs is True:
	fig.savefig(fig_dir + f'R2.6_monkey_directionality_all_bins_dataset_types.pdf', bbox_inches='tight', transparent=True)

### Monkey A Plot EV timescales 

In [ ]:
monkey_name = 'A'
monkey=monkey_name
monkey_stats_path = os.path.join(main_dir, 'results/fig_2',f'monkey_{monkey_name}_stats.pkl')

with open(monkey_stats_path, 'rb') as f:
	monkey_stats = pickle.load(f)

df_monkey = make_monkey_df(monkey_stats, dataset_types=['SNR','RF_thin','RF_large','RS','SNR_spont'])
filt_df_monkey =df_monkey[(df_monkey['Split-half r']>0.8)&(df_monkey.SNR>=2)].reset_index(drop=True)

In [ ]:
# Fig Supp 4B
fontsize=6
activity_types_ordered = ['stimulus','gray screen']
x = 'Activity Type'
hue = 'bin size (ms)'
y = 'EV'
hue_order = [10,25,50,100,200]  # or strings to match your data

plot_df = filt_df_monkey[filt_df_monkey['control_shuffle']==False].copy()

# stats per x label, correction within x (Holm by default)
res = pairwise_within_x_using_your_tests(
    plot_df, x=x, hue=hue, y=y,
    x_order=activity_types_ordered, hue_order=hue_order,
    compare='all',          # each bin vs the first bin (5 ms)
    mc_method='holm',
    perm_t=True, perm_type='ind', hierarchical=False,
    central_tendency='median', num_permutations=10000,
    mouse_or_date='Mouse Name'
)

centers = np.arange(len(activity_types_ordered))
n_h = len(hue_order)
# approximate seaborn offsets for grouped violins
width = 0.83
step = width / n_h if n_h > 0 else 0
start = -width/2 + step/2
offsets = [start + k*step for k in range(n_h)]
hue_to_offset = {h: offsets[i] for i, h in enumerate(hue_order)}


fig, ax = plt.subplots(1,1, figsize=(3,1.3))

sns.violinplot(
    data=plot_df, x=x, y=y, hue=hue,
    order=activity_types_ordered, hue_order=hue_order,
    ax=ax, linewidth=0.5,
    inner_kws={'box_width':2, 'marker':'_', 'markersize':3, 'markeredgewidth':1, 'alpha':0.7},
    cut=0
)

for x_idx, x_i in enumerate(activity_types_ordered):
	for h1, h2 in zip(hue_order[1:], hue_order[:-1]):
		pval = res[x_i]['p_adj'][res[x_i]['pairs'].index((h1,h2))] if (h1,h2) in res[x_i]['pairs'] else (res[x_i]['p_adj'][res[x_i]['pairs'].index((h2,h1))] if (h2,h1) in res[x_i]['pairs'] else None)
		stars =p_to_stars(pval)
		subdf = plot_df[(plot_df[x]==x_i)&(plot_df[hue].isin([h1,h2]))]
		x_center = x_idx + 0.5*(hue_to_offset[h1] + hue_to_offset[h2])
		y_pos = subdf['EV'].max()*0.97	
		if stars == 'n.s.':
			fontsize_=fontsize*0.8
		else:
			fontsize_=fontsize
		ax.text(x_center, y_pos, stars, ha="center", va="top", fontsize=fontsize_)
# annotate_within_x_stars(ax, res, dataset_types_ordered, hue_order, compare='vs_ref', fontsize=fontsize)
sns.despine()

ax.legend(title='Bin size (ms)', loc=(1,0.3), fontsize=fontsize*0.8, title_fontsize=fontsize*0.8)
x_tick_labels = ['Stimulus', 'Gray screen']
ax.set_xticklabels(x_tick_labels, **timescale_main_plot_args['xticklabel_args'])
ax.set_ylabel('EV', **timescale_main_plot_args['ylabel_params'])
ax.set_xlabel('')
ax.tick_params(**timescale_main_plot_args['tick_params'])
ax.legend_.get_frame().set_linewidth(0.5)
data = filt_df_monkey[filt_df_monkey['control_shuffle']==True]['EV']
per_25 = np.percentile(data.values, 25)
per_75 = np.percentile(data.values, 75)
ax.axhspan(per_25, per_75, alpha=0.3, color='blue', label='shuffle\ncontrol IQR',
		linewidth=0,
		)
ax.legend_.remove()
ax.spines[:].set_linewidth(0.3)
if save_figs is True:
	fig.savefig(fig_dir + f'R2.6_monkey_{monkey}_predictions_all_bins_dataset_types.pdf', bbox_inches='tight', transparent=True)

In [ ]:
# perform stats betwen control_shuffle=True and control_shuffle=False
control_shuffle_true_10_ms_gray_screen = filt_df_monkey[(filt_df_monkey['control_shuffle']==True)&(filt_df_monkey['bin size (ms)']==10)&(filt_df_monkey['Activity Type']=='gray screen')]['EV'].to_numpy()
control_shuffle_false_10_ms_gray_screen = filt_df_monkey[(filt_df_monkey['control_shuffle']==False)&(filt_df_monkey['bin size (ms)']==10)&(filt_df_monkey['Activity Type']=='gray screen')]['EV'].to_numpy()

from utils.stats_functions import perm_test_paired
p_value_gray_screen_10_ms = perm_test_paired(control_shuffle_true_10_ms_gray_screen, control_shuffle_false_10_ms_gray_screen)
print(p_value_gray_screen_10_ms)

### Monkey D plot EV timescales 

In [ ]:
monkey_name = 'D'
monkey=monkey_name
monkey_stats_path = os.path.join(main_dir, 'results/fig_2',f'monkey_{monkey_name}_stats.pkl')

with open(monkey_stats_path, 'rb') as f:
	monkey_stats = pickle.load(f)

df_monkey = make_monkey_df(monkey_stats, dataset_types=['SNR','SNR_spont','RS_open'])
filt_df_monkey =df_monkey[(df_monkey['Split-half r']>0.8)&(df_monkey.SNR>=2)].reset_index(drop=True)

In [ ]:
# Fig Supp 4E
fontsize=6
activity_types_ordered = ['stimulus','gray screen','lights off']
x = 'Activity Type'
hue = 'bin size (ms)'
y = 'EV'
hue_order = [10,25,50,100,200]  # or strings to match your data

plot_df = filt_df_monkey[filt_df_monkey['control_shuffle']==False].copy()

# stats per x label, correction within x (Holm by default)
res = pairwise_within_x_using_your_tests(
    plot_df, x=x, hue=hue, y=y,
    x_order=activity_types_ordered, hue_order=hue_order,
    compare='all',          # each bin vs the first bin (5 ms)
    mc_method='holm',
    perm_t=True, perm_type='ind', hierarchical=False,
    central_tendency='median', num_permutations=10000,
    mouse_or_date='Mouse Name'
)

centers = np.arange(len(activity_types_ordered))
n_h = len(hue_order)
# approximate seaborn offsets for grouped violins
width = 0.83
step = width / n_h if n_h > 0 else 0
start = -width/2 + step/2
offsets = [start + k*step for k in range(n_h)]
hue_to_offset = {h: offsets[i] for i, h in enumerate(hue_order)}


# fig, ax = plt.subplots(1,1, figsize=timescale_main_plot_args['figsize'])
fig, ax = plt.subplots(1,1, figsize=(3.5,1.3))

sns.violinplot(
    data=plot_df, x=x, y=y, hue=hue,
    order=activity_types_ordered, hue_order=hue_order,
    ax=ax, linewidth=0.5,
    inner_kws={'box_width':2, 'marker':'_', 'markersize':3, 'markeredgewidth':1, 'alpha':0.7},
    cut=0
)

for x_idx, x_i in enumerate(activity_types_ordered):
	for h1, h2 in zip(hue_order[1:], hue_order[:-1]):
		pval = res[x_i]['p_adj'][res[x_i]['pairs'].index((h1,h2))] if (h1,h2) in res[x_i]['pairs'] else (res[x_i]['p_adj'][res[x_i]['pairs'].index((h2,h1))] if (h2,h1) in res[x_i]['pairs'] else None)
		stars =p_to_stars(pval)
		subdf = plot_df[(plot_df[x]==x_i)&(plot_df[hue].isin([h1,h2]))]
		x_center = x_idx + 0.5*(hue_to_offset[h1] + hue_to_offset[h2])
		y_pos = subdf['EV'].max() + 0.03
		if stars == 'n.s.':
			fontsize_=fontsize*0.8
		else:
			fontsize_=fontsize
		ax.text(x_center, y_pos, stars, ha="center", va="top", fontsize=fontsize_)
# annotate_within_x_stars(ax, res, dataset_types_ordered, hue_order, compare='vs_ref', fontsize=fontsize)
sns.despine()

ax.legend(title='Bin size (ms)', loc=(1,0.3), fontsize=fontsize*0.8, title_fontsize=fontsize*0.8)
x_tick_labels = ['Stimulus', 'Gray screen', 'Lights off']
ax.set_xticklabels(x_tick_labels, **timescale_main_plot_args['xticklabel_args'])
ax.set_ylabel('EV', **timescale_main_plot_args['ylabel_params'])
ax.set_xlabel('')
ax.tick_params(**timescale_main_plot_args['tick_params'])
ax.legend_.get_frame().set_linewidth(0.5)
data = filt_df_monkey[filt_df_monkey['control_shuffle']==True]['EV']
per_25 = np.percentile(data.values, 25)
per_75 = np.percentile(data.values, 75)
ax.axhspan(per_25, per_75, alpha=0.3, color='blue', label='shuffle\ncontrol IQR',
		linewidth=0,
		)
ax.spines[:].set_linewidth(0.3)
ax.legend_.remove()
if save_figs is True:
	fig.savefig(fig_dir + f'R2.6_monkey_{monkey}_predictions_all_bins_dataset_types.pdf', bbox_inches='tight', transparent=True)

### Monkey A plot directionality across timescales

In [ ]:
# Fig Supp 4D
monkey_name = 'A'
monkey=monkey_name
# import monkey states from fig4
monkey_stats_path = os.path.join(main_dir, 'results/fig_3',f'monkey_{monkey_name}_stats.pkl')
with open(monkey_stats_path, 'rb') as f:
	monkey_stats = pickle.load(f)

df_monkey_directionality = make_monkey_df_directionality(monkey_stats, dataset_types=['SNR','RF_thin','RF_large','SNR_spont'], verbose=False)
filt_df_monkey_directionality = df_monkey_directionality[(df_monkey_directionality.control_shuffle==False)].reset_index(drop=True)


fontsize=6
#'#136a66','#72BEB7','#B6E3DF','#a85959','#EDAEAE', '#f6d6d6
areas_palette = ['#72BEB7','#EDAEAE']
activity_types_ordered = ['stimulus','gray screen']
x_tick_labels = ['5','10','25','50','100','200']
x = 'bin size (ms)'
hue = 'Direction'
neuron_property = 'EV'
hue_order=['V1→V4','V4→V1']
fig, axes = plt.subplots (ncols=2, figsize=(3.5,1.3), sharey=True)
axes = axes.flatten()
# plot violin plot and change inner quartile widths using the inner_kws argument
for a, (act_type, ax) in enumerate(zip(activity_types_ordered, axes)):
	mini_df = df_monkey_directionality[df_monkey_directionality['Activity Type']==act_type]
	order = [int(i) for i in mini_df[x].unique()]
	_, _, pvals = plot_activitytype_by_direction_densities(
		mini_df, neuron_property=neuron_property, x=x, hue=hue,
		neuron_property_label="EV", ax=ax,
		order=order, hue_order=hue_order,
		fontsize=fontsize,
		linewidth=0,
	)
	ax.set_xlabel('Bin size (ms)', fontsize=fontsize, labelpad=0)
	ax.tick_params(axis='y', labelsize=fontsize)
	ax.set_title(act_type.capitalize(), fontsize=fontsize, y=0.9)
	ax.legend_.remove()

plt.subplots_adjust(wspace=0.05, hspace=0.5)
if save_figs is True:
	fig.savefig(fig_dir + f'R2.6_monkey_{monkey_name}_directionality_all_bins_dataset_types.pdf', bbox_inches='tight', transparent=True)